In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Problem statement and motivation

Let us fix some notation:
 - $\mathcal{S}$ - a finite set of hidden states (denote $|\mathcal{S}| = k$)
 - $\mathcal{X}$ - an alphabet of observed symbols
 - $\lbrace \mathcal{T}^{(x)} \rbrace$ - a set of sub-stochastic matrices indexed by $x \in \mathcal{X}$ where $\mathcal{T}^{(x)}_{ij} = \Pr (S_{t+1} = j, X_t = x \mid S_t = i)$
 - $\mu$ - an initial distribution of hidden states, or a stationary distribution if it exists and we assume that we are observing $X_1^n$ after stationary distribution being achieved.

Unifilarity condition imposes some sparsity restrictions on $\lbrace \mathcal{T}^{(x)} \rbrace$, namely each matrix $\mathcal{T}^{(x)}$ has at most one non-zero entry in each row. Therefore we can respresent $\lbrace \mathcal{T}^{(x)} \rbrace$ as a tuple $(\delta, \epsilon)$ where:
 - $\delta : \mathcal{S} \times \mathcal{X} \to \mathcal{S}$ is a deterministic function of state-to-state transition
 - $\epsilon(x | s)$ are emission probability ditributions for each state.

Given an observed data $x_1^n := x_1, x_2, \dots x_n$ we desire to infer HMM structure $\theta = (k, \delta, \epsilon)$ and possibly an initial distribution $\mu$ over the hidden states $\mathcal{S}$.

Note that $k$ and $\delta$ are discrete objects, while $\epsilon$ is a continuous object. Let us denote $\tau = (k, \delta)$ and call it a *topology* of uHMM. When, additionally to observed $x_1^n$, we know also the toplogy $\tau$ and an initial state $s_1$, then immediately we know the whole trajectory of hidden states $s_1^n := s_1, s_2, \dots, s_n$. This is in contrast with general theory of HMM (without unifilarity restriction) where multiple hidden trajectories are possible and we can infer only the distribution over these trajectories.

Knowledge of the topology $\tau$ brings one more benefit: we can estimate empirical emission probabilities $\hat{\epsilon}(x | s)$ from the observed emissions $x_1^n$. (Is it consistent via CLT?)

However, the inference of $\tau$ itself imposes certain difficulties:
 - set of all possible topologies $\tau$ is discrete
 - it is computationally infeasible to check all possible combinations of $k$ and $\delta$, even if we assume upper bound $k_{\max}$.

 Nonetheless we are interested in estimating $\hat{\tau}$ from the observed $x_1^n$. For instance, in a standard MLE framework our problem statement would be $\hat{\tau} = \arg \max_\tau \Pr (x_1^n \mid \tau = (k, \delta))$. Order estimation $\hat{k}$ has been long standing topic in the literature.

## Order estimation

Below thoughts are based on results from [1] where the authors consider consistent estimation of the number of hidden states in a state-emitting HMM. They employ two techniques: *Shtarkov normalized maximum likelihood* and *Krichevsky-Trokimov mixtures*.

In the following we will narrow down their framework to a specific class of hidden Markov models, namely *unifilar edge-emitting HMM*.

The goal is to estimate $k = |\mathcal{S}|$ given a word $x_1^n \in \mathcal{X}^n$.

Probability of observing a word $x_1^n$ is given by
$$
\Pr (x_1^n) = \mu^\intercal \prod_{i=1}^n \mathcal{T}^{(x_i)} \mathbf{1}
$$

In an unifilar setting, once we know an initial state $s_1$, the trajectory of hidden states $s_1^n$ is determined by the observed sequence $x_1^n$:
$$
\Pr \left( x_1^n \mid s_1, \lbrace p_i (\cdot) \rbrace \right) = \prod_{i=1}^n p_{s_i} (x_i), \quad s_{i+1} = \delta (s_i, x_i)
$$
for a deterministic function $\delta : \mathcal{S} \times \mathcal{X} \to \mathcal{S}$ and
$$
p_{s_i} (x) = \sum_{j} T_{ij}^{(x)} = T_{i, \delta(s_i, x)}^{(x)}
$$

## Inspirations from coding theory

Let us remind some facts from Cover & Thomas.

Given a sequence of random variables $X_1, X_2, \dots , X_n$ over the alphabet $\mathcal{X}$ one wants to design an optimal coding $C: \mathcal{X} \to \lbrace 0 , 1 \rbrace^*$. Optimal means, that the expected length $\mathbb{E} [ l(X_1), l(X_2), \dots , l(X_n) ]$ is minimal.

A code $C$ is uniquelly decodable when it's extension $C^*$ has the following property:
$$
x_1^n \neq y_1^n \implies C(x_1^n) \neq C(y_1^n) \quad \text{where} \ C(x_1^n) = C(x_1) C(x_2) \dots C(x_n)
$$

Kraft-McMillan theorem states that for an uniquely decodable code
$$
\sum_{x \in \mathcal{X}} 2^{-l(x)} \leq 1
$$

Think of $2^{-l(\cdot)}$ as a (sub)probability measure. It has clear inutitive meaning: more frequent words $x \in \mathcal{X}$ are assigned to shorter codewords $C(x)$. If $Q$ denotes probability of $C(x_1^n)$, i.e. $Q = \sum_{i=1}^n 2^{-l(x_i)}$, then $l(x_1^n) = -\log_2 Q$. In order to maximize probability, we need to minimize length of the code.

Inspired by Kraft-McMillan theorem, they[1] find optimal $k$ by minimizing average number of bits required to compress $X_1^n$ via uniquely decodable code and adding a pentalty for a complexity:
$$
\hat{k} = \arg \min_k (-\log_2 Q_k^n (X_1^n) + \lambda (n,k))
$$

Why a well-choosen penalty $\lambda$ is essential to guarantee consistency?
 - if a penalty is too small, we risk overestimating $k$
 - if a penalty is too large, we risk underestimating $k$.

To guarantee a consistency, the penalty must offset
$$
\sup_{\theta \in \Theta^k} L^n (\theta, X_1^n) - L^n (\theta_{true}, X_1^n)
$$

How to choose a coding distribution $Q_k^n (\cdot)$ for the words $x_1^n \in \mathcal{X}^n$?

### Shtarkov NML

$$
NML_k^n (x_1^n) = \frac{\sup_{\theta \in \Theta^k} \mathbb{P}_\theta (x_1^n)}{C(k,n)}
$$

where
$$
C(k,n) = \sum_{x_1^n \in \mathcal{X}^n} \sup_{\theta \in \Theta^k} \mathbb{P}_\theta (x_1^n)
$$

It is evident that $C(k,n)$ may be difficult to calcualte for large $n$. In [1, Lemma 8] an upper bound is derived for $C(k,n)$. A special case, where $C(k,n) = C(k)$, is a BIC estimator.

**Open question: can we find a tighter upper bound for $C(k,n)$ if we narrow down the scope to unifilar HMM?**

### Krichevsky-Trokimov mixture

KT is a Bayesian approach, where a priori dsitribution is Dirichlet and then we take an expectation over the parameters (more details and formulas can be found in [1])
$$
KT_k^n (x_1^n) = \mathbb{E}_{\nu_k} \left[ \mathbb{P}_{\mu, \theta} (x_1^k) \right]
$$

where $\nu_k$ is a density of Dirichlet distribution on $\Theta^k$.

We will be seeking not only $k$ as in [1], but $(k, \delta)$. For a fixed $k$ we choose candidate topologies $\mathcal{D}_k = \lbrace \delta \rbrace$

## 3-stage consistent estimation of unifilar HMM

Unifilarity simplifies inference. For a parameter set $\theta = (k, \delta, \epsilon)$

- Stage 1. Estimate $\hat{k}$ - consistent estimator is proved in [2]
- Stage 2. Estimate $\hat{\delta}$ - problem to be solved
- Stage 3. Estimate $\hat{\epsilon}$ - estimated from empirical frequencies assuming initial state $s_1$ is known. Under ergodicity assumption CLT guarantees consistency.

## References

1. Gassiat, E. & Boucheron, S. Optimal error exponents in hidden markov models order estimation. IEEE Trans. Inform. Theory 49, 964–980 (2003).
2. Dębowski, Ł. A Refutation of Finite-State Language Models through Zipf’s Law for Factual Knowledge. Entropy 23, 1148 (2021).
